## MC Gesture Recognition

In [0]:
import pandas as pd

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectFromModel

from joblib import dump, load

In [2]:
! wget -Nq 'https://raw.githubusercontent.com/renilJoseph/MC_Assignment_GestureIdentification/master/agg_data.csv'
#! wget -Nq 'https://raw.githubusercontent.com/renilJoseph/MC_Assignment_GestureIdentification/master/merged_data.csv'
df = pd.read_csv('agg_data.csv')
#Check num columns
print(df.shape)
df.head()

(415, 314)


,vid_id,score_overall_mean,score_overall_std,score_overall_amin,score_overall_amax,score_overall_kurt,score_overall_skew,nose_score_mean,nose_score_std,nose_score_amin,nose_score_amax,nose_score_kurt,nose_score_skew,nose_x_mean,nose_x_std,nose_x_amin,nose_x_amax,nose_x_kurt,nose_x_skew,nose_y_mean,nose_y_std,nose_y_amin,nose_y_amax,nose_y_kurt,nose_y_skew,leftEye_score_mean,leftEye_score_std,leftEye_score_amin,leftEye_score_amax,leftEye_score_kurt,leftEye_score_skew,leftEye_x_mean,leftEye_x_std,leftEye_x_amin,leftEye_x_amax,leftEye_x_kurt,leftEye_x_skew,leftEye_y_mean,leftEye_y_std,leftEye_y_amin,...,rightKnee_y_amax,rightKnee_y_kurt,rightKnee_y_skew,leftAnkle_score_mean,leftAnkle_score_std,leftAnkle_score_amin,leftAnkle_score_amax,leftAnkle_score_kurt,leftAnkle_score_skew,leftAnkle_x_mean,leftAnkle_x_std,leftAnkle_x_amin,leftAnkle_x_amax,leftAnkle_x_kurt,leftAnkle_x_skew,leftAnkle_y_mean,leftAnkle_y_std,leftAnkle_y_amin,leftAnkle_y_amax,leftAnkle_y_kurt,leftAnkle_y_skew,rightAnkle_score_mean,rightAnkle_score_std,rightAnkle_score_amin,rightAnkle_score_amax,rightAnkle_score_kurt,rightAnkle_score_skew,rightAnkle_x_mean,rightAnkle_x_std,rightAnkle_x_amin,rightAnkle_x_amax,rightAnkle_x_kurt,rightAnkle_x_skew,rightAnkle_y_mean,rightAnkle_y_std,rightAnkle_y_amin,rightAnkle_y_amax,rightAnkle_y_kurt,rightAnkle_y_skew,type
0,0,0.924542,0.009734,0.903812,0.941532,-1.069618,-0.094263,0.999623,0.000374,0.996954,0.999938,25.948714,-4.493033,-594.306323,14.186974,-616.144126,-562.105126,-0.885329,0.511757,264.755487,20.978431,211.232493,290.606517,-1.174224,-0.453479,0.999975,0.000019,0.999905,0.999993,4.710506,-2.142918,-561.288788,12.813013,-587.047587,-532.455177,-0.600858,0.196490,226.044462,21.165453,169.355469,...,1078.129096,0.212720,-0.396384,0.787298,0.101603,0.557715,0.976936,-0.677431,-0.108322,-613.887527,18.823947,-636.811761,-561.523626,-0.894968,0.679417,1242.511694,23.277902,1176.998478,1270.500644,-0.488747,-0.848680,0.503132,0.097474,0.307590,0.703325,-1.038708,0.027250,-581.705119,30.121973,-670.526368,-480.299357,2.870869,1.043343,1226.472542,43.362531,1029.246450,1261.348705,11.256338,-3.138566,COMMUNICATE
1,1,0.975678,0.005077,0.960819,0.984514,-0.734473,-0.179723,0.999858,0.000107,0.999439,0.999950,2.977167,-1.791727,-731.476389,13.219975,-756.349668,-704.355076,-1.133285,0.009233,319.022872,4.782174,308.214171,329.399762,-0.431989,-0.370869,0.999967,0.000015,0.999921,0.999987,0.395981,-0.986457,-694.591843,13.563072,-739.151095,-668.823398,0.161852,-0.505561,291.499942,4.298494,280.883990,...,1383.397267,-0.016273,0.473127,0.953317,0.013011,0.908230,0.973762,0.756638,-0.981520,-562.923621,11.264662,-588.637394,-536.127591,-0.674126,-0.034706,1708.417425,5.464382,1698.382491,1721.315982,-0.674021,0.470730,0.832059,0.065854,0.659138,0.945943,-0.792795,-0.208521,-867.219755,14.044414,-899.290319,-839.513740,-0.915625,-0.052779,1676.101800,6.928823,1660.215210,1698.115783,1.327429,0.784031,COMMUNICATE
2,2,0.969402,0.007790,0.925032,0.979215,8.900604,-2.427046,0.999809,0.000056,0.999642,0.999909,0.302725,-0.712581,-309.534332,6.085619,-329.169425,-295.618711,0.262002,-0.565722,152.894271,3.340848,148.956943,161.953866,0.521890,1.145198,0.999954,0.000027,0.999837,0.999990,3.185505,-1.648614,-296.091557,6.186804,-314.835610,-279.476360,0.204055,-0.090061,141.214723,3.002162,137.480935,...,510.556970,-0.938555,0.509783,0.990677,0.003192,0.978772,0.996258,0.950654,-0.825282,-298.352813,5.744417,-311.721027,-286.185913,-0.538606,-0.319827,596.747069,3.058837,588.309285,604.728538,-0.243445,0.119347,0.996584,0.001230,0.993010,0.998624,0.024556,-0.626827,-325.350050,5.433540,-339.798101,-313.614149,-0.460369,-0.396154,600.748579,2.293214,596.936117,605.882417,-0.814317,0.286643,COMMUNICATE
3,3,0.752685,0.007609,0.735594,0.772238,-0.473247,0.097008,0.999793,0.000107,0.999534,0.999946,-0.819191,-0.583172,-486.433177,5.213434,-499.367127,-473.352346,-0.140897,-0.325264,247.872443,3.982467,237.873535,254.670767,0.030412,-0.386125,0.

In [0]:
assert df.shape[1] == 314

In [4]:
labels = ['buy', 'fun', 'hope', 'really', 'communicate', 'mother']
df = df[df['type'].isin(map(str.upper, labels))]
df.shape

(415, 314)

In [5]:
le = preprocessing.LabelEncoder()
# Converting string labels into numbers.
labels=le.fit_transform(df['type'])
features = df.drop(['type', 'vid_id'], axis = 1)

X_train = features
y_train = labels
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'BUY': 0, 'COMMUNICATE': 1, 'FUN': 2, 'HOPE': 3, 'MOTHER': 4, 'REALLY': 5}


In [0]:
#feature_list = list(features.columns)
# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, random_state = 1, stratify=labels, shuffle=True)

In [7]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
#print('Testing Features Shape:', X_test.shape)
#print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (415, 312)
Training Labels Shape: (415,)


## Feature Selection

In [9]:
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X_train, y_train)

model = SelectFromModel(clf, prefit=True)
X_train_dr = model.transform(X_train)
#X_test_dr = model.transform(X_test)

dump(model, 'feat_selection.joblib')
print(f'New shape : {X_train_dr.shape}')

New shape : (415, 96)


In [0]:
scoring = ['accuracy']

In [11]:
rf_clf = RandomForestClassifier(n_estimators=50, random_state=12)
scores = cross_validate(rf_clf, X_train_dr, y_train, cv=5, scoring=scoring)
scores['test_accuracy']

array([0.75294118, 0.75903614, 0.79518072, 0.84146341, 0.79268293])

In [12]:
lr_clf = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000)
scores = cross_validate(lr_clf, X_train_dr, y_train, cv=5, scoring=scoring)
scores['test_accuracy']

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


array([0.83529412, 0.81927711, 0.8313253 , 0.81707317, 0.85365854])

In [13]:
gbm_clf = GradientBoostingClassifier(random_state=12)
scores = cross_validate(gbm_clf, X_train_dr, y_train, cv=5, scoring=scoring)
scores['test_accuracy']

array([0.70588235, 0.78313253, 0.79518072, 0.7804878 , 0.81707317])

In [14]:
clf1 = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
clf2 = ExtraTreesClassifier(n_estimators=100, random_state=17)
clf3 = LogisticRegression(solver='newton-cg', multi_class='multinomial', penalty='none', max_iter=200)#SVC(gamma='scale', kernel='rbf', probability=True)
eclf = VotingClassifier(estimators=[('dt', clf1), ('knn', clf2), ('svc', clf3)],
                        voting='soft', weights=[3, 2, 2])
scores = cross_validate(eclf, X_train_dr, y_train, cv=5, scoring=scoring)
scores['test_accuracy']


array([0.83529412, 0.85542169, 0.87951807, 0.86585366, 0.90243902])

In [16]:
clfs = {'rf_clf' : rf_clf,
        'xgb_clf' : gbm_clf,
        'lr_clf' : lr_clf,
        'vot_clf' : eclf
        }

for clf_name in clfs:
    print(f'Fitting {clf_name}')
    clfs[clf_name].fit(X_train_dr, y_train)
    #score = clfs[clf_name].score(X_test_dr, y_test)
    #print(f'Test accuracy : {score:.3f}')
    dump(clfs[clf_name], f'{clf_name}.joblib') 

Fitting rf_clf
Fitting xgb_clf
Fitting lr_clf


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Fitting vot_clf
